# FEM Modelling for STEDIN Transformer

In this part, the conducitvity of the core material is modelled as 0. Therefore, there areno eddy current effects.

## Import Packages

In [1]:
using gmsh
using LinearAlgebra
using LaTeXStrings
include("../../../lib/Makie_Plots.jl");
using .Makie_Plots
include("../../../lib/FEM_VoltageFed_Tri_1e.jl");
using .FEM_VoltageFed_Tri_1e
include("../../../lib/Mesh_Data_stedin.jl");
using .Mesh_Data_stedin
include("../../../lib/Save_VTK.jl");
using .Save_VTK

## Julia code

In [2]:
gmsh.initialize()

## Read mesh from file
gmsh.open("../mesh/stedin_transformer.msh")

mesh_data = get_mesh_data_tri_1e(gmsh)

Ip = 17.54;   # Primary peak phase current
Is = 777.62;  # Secondary peak phase current
Vp = 15202.40;   # Primary peak phase voltage
Vs = 342.93;     # Secondary peak phase voltage
Np = 266;
Ns = 6;
Rext = 0;
Lext = 0;

omega = 2*pi*50;  # Frequency

# HV winding dimensions (all phases left/right are identical)
wwhv = 3e-2;
hwhv = 74e-2;
mwhv = 14.75e-2;
Awhv = wwhv * hwhv;

# LV winding dimensions (all phases left/right are identical)
wwlv = 2e-2;
hwlv = 74e-2;
mwlv = 11.25e-2;
Awlv = wwlv * hwlv;

# Calculate turn density in the windings
Tp = Np / Awhv;
Ts = Ns / Awlv;

# Source turn density T
# One term for each of the windings, with a positive and negative part
sourcefunction(group_id) = [Ts*(1*(group_id==9)) Ts*(1*(group_id==10)) Ts*(1*(group_id==11)) Ts*(1*(group_id==12)) Ts*(1*(group_id==13)) Ts*(1*(group_id==14))];
sourceperelement = map(sourcefunction, mesh_data.e_group);

# Source voltage V
# One term for each of the windings, with a positive and negative part
coil_voltage = [Vs*exp(1im*-2pi/3); -Vs*exp(1im*-2pi/3); Vs; -Vs; Vs*exp(1im*2pi/3); -Vs*exp(1im*2pi/3)];

# External resistance Rext
# One term for each of the windings, with a positive and negative part
ext_resistance = Rext*[1; 1; 1; 1; 1; 1];

# External resistance Rext
# One term for each of the windings, with a positive and negative part
ext_inductance = Lext*[1; 1; 1; 1; 1; 1];

# Relative permeability model
mu0 = 4e-7 * pi;
mur = 2.5e4;       # Relative permeability of the core
reluctivityfunction(group_id) = (1 / mu0) + (1/(mu0*mur) - 1/mu0) * (group_id == 2)
reluctivityperelement = map(reluctivityfunction, mesh_data.e_group);

# Conductivity
conductivityfunction(group_id) = 0;
conductivityperelement = map(conductivityfunction, mesh_data.e_group);

# Handle the boundary conditions
bnd_node_ids, _ = gmsh.model.mesh.getNodesForPhysicalGroup(1, 1);

# Calculate the vector potential
u = fem(mesh_data, sourceperelement, reluctivityperelement, conductivityperelement, omega, bnd_node_ids, coil_voltage, ext_resistance, ext_inductance);

# Post-process for magnetic field and current density
Bx, By, B, Hx, Hy, H, mag_energy = post_process(mesh_data, u[1:mesh_data.nnodes], reluctivityperelement);

# Contour plot of the magnetic flux density
plot_surface_tri_1e(norm.(B), "stedin_transformer_B_plot_voltagefed", gmsh)
# Contour plot of the magnetic field strength
plot_surface_tri_1e(norm.(H), "stedin_transformer_H_plot_voltagefed", gmsh)
# Contour plot of the magnetic energy
plot_surface_tri_1e(norm.(mag_energy), "stedin_transformer_mag_energy_plot_voltagefed", gmsh)

# Save as VTK file for Paraview visualization
save_vtk(mesh_data, imag(u[1:mesh_data.nnodes]), norm.(B), norm.(H), norm.(mag_energy), reluctivityperelement, "stedin_transformer_voltagefed")

gmsh.finalize()

Info    : Reading '../mesh/stedin_transformer.msh'...
Info    : 168 entities
Info    : 10385 nodes
Info    : 20768 elements
Info    : Done reading '../mesh/stedin_transformer.msh'


### STEDIN Transformer : Magnetic field density
<img src="../img/stedin_transformer_B_plot_voltagefed.png">

### STEDIN Transformer : Magnetic field strength
<img src="../img/stedin_transformer_H_plot_voltagefed.png">

### STEDIN Transformer : Magnetic energy
<img src="../img/stedin_transformer_mag_energy_plot_voltagefed.png">

In [3]:
display(maximum(norm.(B)))

4.08962164994436